# Optional Add-on Module 5: Integrating Pandas with Databases, APIs & Big Data Systems

This module covers how to move Pandas beyond in-memory data processing — connecting it to **SQL databases, REST APIs, and distributed formats** for scalable data engineering.

By the end, you will learn to:
- Read/write efficiently to **SQL databases**.
- Fetch and process live data from **REST APIs**.
- Work with **Parquet**, **Feather**, and **Arrow** formats for large-scale data.
- Integrate **DuckDB** and **BigQuery** for analytics at scale.

Let's dive in! 🚀

## 1. Working with SQL Databases

Pandas integrates tightly with SQLAlchemy, allowing direct I/O with databases like MySQL, PostgreSQL, SQLite, etc.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Create a SQLite database (for demo)
engine = create_engine('sqlite:///sales.db', echo=False)

# Create sample data
sales = pd.DataFrame({
    'product': ['A', 'B', 'C', 'D'],
    'region': ['North', 'East', 'West', 'South'],
    'revenue': [10000, 15000, 12000, 17000]
})

# Write to SQL
sales.to_sql('sales_data', con=engine, if_exists='replace', index=False)

# Read from SQL
df_sql = pd.read_sql('SELECT * FROM sales_data', con=engine)
df_sql

✅ **Tip:** For large datasets, use `chunksize` in `read_sql` to stream results efficiently.

```python
for chunk in pd.read_sql('SELECT * FROM big_table', con=engine, chunksize=10000):
    process(chunk)
```

## 2. Interacting with REST APIs

APIs provide real-time data feeds. You can use `requests` to fetch JSON and easily convert it into Pandas DataFrames.

In [ ]:
import requests

# Example: Fetch cryptocurrency market data from CoinDesk API
url = 'https://api.coindesk.com/v1/bpi/currentprice.json'
response = requests.get(url)
data = response.json()

# Convert to DataFrame
df_api = pd.DataFrame.from_dict(data['bpi'], orient='index')
df_api

### Real-World Example: Weather Forecast API
You can use public weather APIs like Open-Meteo to fetch daily weather summaries.

```python
url = 'https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41&daily=temperature_2m_max&timezone=auto'
data = requests.get(url).json()
df_weather = pd.DataFrame({
    'date': data['daily']['time'],
    'max_temp': data['daily']['temperature_2m_max']
})
```

## 3. Working with Parquet, Feather, and Arrow

Columnar formats like Parquet and Feather are optimized for performance and interoperability with Spark, Arrow, and Dask.

In [ ]:
df = pd.DataFrame({
    'user_id': range(1, 6),
    'score': [85, 90, 76, 88, 92]
})

# Save as Parquet
df.to_parquet('scores.parquet')

# Load back
df_parquet = pd.read_parquet('scores.parquet')
df_parquet.head()

✅ **Performance Comparison:**
| Format | Compression | Speed | Use Case |
|---------|--------------|--------|-----------|
| CSV | None | Slow | Simple exchange |
| Parquet | Yes | Fast | Analytics, Big Data |
| Feather | Optional | Very Fast | In-memory pipeline |
| Arrow | Yes | Very Fast | Cross-language data sharing |

## 4. DuckDB Integration

DuckDB is an **in-process analytical database** that works natively with Pandas and Parquet — great for query-heavy workloads.

In [ ]:
!pip install -q duckdb
import duckdb

# Query Pandas DataFrame directly using DuckDB
query_result = duckdb.query('SELECT AVG(revenue) AS avg_rev FROM sales').to_df()
query_result

✅ **Advantages:**
- No external server — runs locally.
- Queries large Parquet files directly.
- Integrates with Pandas, Arrow, and Polars seamlessly.

## 5. Integration with BigQuery / Cloud Data Warehouses

Pandas can push queries and pull results from BigQuery, Snowflake, or Redshift using their official Python clients.

In [ ]:
# Example (Google BigQuery)
'''
from google.cloud import bigquery
client = bigquery.Client()
query = 'SELECT name, SUM(number) as total FROM `bigquery-public-data.usa_names.usa_1910_2013` GROUP BY name LIMIT 5'
df_bq = client.query(query).to_dataframe()
df_bq.head()
'''

## 6. Real-World Problems

### Problem 1: Build a Data Pipeline from API to Database
**Goal:** Fetch daily COVID-19 case data from an API, clean it with Pandas, and store it in SQLite.

Steps:
1. Request JSON data using `requests`.
2. Convert into a normalized Pandas DataFrame.
3. Save to SQL using `.to_sql()`.
4. Visualize top-5 countries by case count.

### Problem 2: Querying and Aggregating Parquet Data
**Goal:** Analyze millions of sales records stored in Parquet files using DuckDB and Pandas.

Steps:
1. Use DuckDB to query aggregated data directly from Parquet.
2. Convert query results into Pandas.
3. Plot sales trends using Matplotlib.

```python
query = "SELECT region, SUM(revenue) as total_rev FROM 'sales.parquet' GROUP BY region"
df_agg = duckdb.query(query).to_df()
df_agg.plot(kind='bar', x='region', y='total_rev', title='Revenue by Region')
```

## 7. Best Practices

✅ **Best Practices:**
- Always use **SQLAlchemy** for flexible DB connections.
- Prefer **Parquet** for high-volume analytics.
- Cache API responses locally to reduce rate limits.
- Offload heavy queries to **DuckDB** or **BigQuery**.

⚠️ **Pitfalls:**
- Avoid reading entire API payloads if pagination exists.
- Don’t store sensitive credentials in plain text.
- Ensure schema consistency between Pandas and SQL types.

## 🧩 Challenge Exercise

**Task:**
1. Pull public stock data for 10 tickers using a REST API.
2. Combine and clean the dataset with Pandas.
3. Store it in SQLite and query top-3 gainers using SQL.
4. Save cleaned dataset to Parquet.

_Hint:_ Use `yfinance` or `AlphaVantage` API and explore DuckDB queries over the resulting Parquet files.